In [1]:
#@test {"skip": true}

!pip install --quiet --upgrade tensorflow-federated

In [2]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/LAB3-20221109T160610Z-001/LCL-FullData/Cluster9data.csv')

In [6]:
# dividing the two year data into 24 parts (each 1 month data)
data['DateTime'] = pd.to_datetime(data['DateTime'])

In [7]:
# Filter the DataFrame to include readings from Jan 01, 2012, to Dec 31, 2012
start_date1 = pd.to_datetime('2012-01-01')
end_date1 = pd.to_datetime('2012-12-31')
filtered_data1 = data[(data['DateTime'] >= start_date1) & (data['DateTime'] <= end_date1)]

# Filter the DataFrame to include readings from Jan 01, 2013, to Dec 31, 2013
start_date2 = pd.to_datetime('2013-01-01')
end_date2 = pd.to_datetime('2013-12-31')
filtered_data2 = data[(data['DateTime'] >= start_date2) & (data['DateTime'] <= end_date2)]

In [8]:
# Select 5 unique LCLids
lclid_list1 = filtered_data1['LCLid'].unique()
selected_lclids1 = lclid_list1[:20]

# Filter data for the selected LCLids
f_data1 = filtered_data1[filtered_data1['LCLid'].isin(selected_lclids1)]

# Select 5 unique LCLids
lclid_list2 = filtered_data2['LCLid'].unique()
selected_lclids2 = lclid_list2[:20]

# Filter data for the selected LCLids
f_data2 = filtered_data2[filtered_data2['LCLid'].isin(selected_lclids2)]

In [9]:
datan1 = f_data1.copy()
datan1['KWH/hh'] = datan1['KWH/hh'].astype(np.float32)
datan2 = f_data2.copy()
datan2['KWH/hh'] = datan2['KWH/hh'].astype(np.float32)

datan1 = datan1.drop('cluster', axis=1)
datan1 = datan1.drop('stdorToU', axis=1)
datan2 = datan2.drop('cluster', axis=1)
datan2 = datan2.drop('stdorToU', axis=1)

In [10]:
datan1.reset_index(drop=True, inplace=True)
datan2.reset_index(drop=True, inplace=True)

In [15]:
datan1['DateTime'] = pd.to_datetime(datan1.DateTime).dt.tz_localize(None)
for i in range(len(datan1)):
  datan1['DateTime'][i]=datan1['DateTime'][i].timestamp()

<ipython-input-15-b59380e28e58>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datan1['DateTime'][i]=datan1['DateTime'][i].timestamp()


In [11]:
datan2['DateTime'] = pd.to_datetime(datan2.DateTime).dt.tz_localize(None)
for i in range(len(datan2)):
  datan2['DateTime'][i]=datan2['DateTime'][i].timestamp()

<ipython-input-11-04c0d2c83489>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datan2['DateTime'][i]=datan2['DateTime'][i].timestamp()


In [16]:
datan1['DateTime'] = datan1['DateTime'].astype(np.float32)
datan2['DateTime'] = datan2['DateTime'].astype(np.float32)

In [17]:
# Sort the data by 'LCLid' and 'DateTime'
datan1.sort_values(['LCLid', 'DateTime'], inplace=True)
datan2.sort_values(['LCLid', 'DateTime'], inplace=True)

In [18]:
# Define the client window dataset function for a specific LCLid
def create_client_dataset_for_LCLid(client_data, window_size, step_size):
    client_windows = []
    client_targets = []
    num_readings = len(client_data)

    # Iterate over the readings using the sliding window
    for i in range(0, num_readings - window_size, step_size):
        window_start = i
        window_end = i + window_size - 1
        prediction_index = window_end + step_size

        # Extract the window and the prediction target
        window = client_data.iloc[window_start:window_end + 1]['KWH/hh'].values
        target = client_data.iloc[prediction_index]['KWH/hh']

        client_windows.append(window)
        client_targets.append(target)

    # Create an ordered dictionary with 'x' and 'y' keys
    ordered_dict = collections.OrderedDict()
    ordered_dict['x'] = tf.stack(client_windows)
    ordered_dict['y'] = tf.expand_dims(client_targets, axis=-1)


    return ordered_dict

In [19]:
window_size = 336
step_size = 1

# Filter the dataframe for the specific LCLid
example_LCLid = datan1['LCLid'].unique()[3]
clientyy_data = datan1[datan1['LCLid'] == example_LCLid]

# Create the client dataset for the specific LCLid
example_client_dataset = create_client_dataset_for_LCLid(clientyy_data, window_size, step_size)

print("Client dataset for LCLid", example_LCLid)
print(example_client_dataset)

Client dataset for LCLid MAC000056
OrderedDict([('x', <tf.Tensor: shape=(17185, 336), dtype=float32, numpy=
array([[0.052, 0.051, 0.051, ..., 0.214, 0.19 , 0.223],
       [0.051, 0.051, 0.028, ..., 0.19 , 0.223, 0.219],
       [0.051, 0.028, 0.   , ..., 0.223, 0.219, 0.074],
       ...,
       [0.043, 0.009, 0.   , ..., 0.   , 0.   , 0.   ],
       [0.009, 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.041]], dtype=float32)>), ('y', <tf.Tensor: shape=(17185, 1), dtype=float32, numpy=
array([[0.219],
       [0.074],
       [0.021],
       ...,
       [0.   ],
       [0.041],
       [0.011]], dtype=float32)>)])


In [20]:
NUM_EPOCHS = 5
BATCH_SIZE = 12
SHUFFLE_BUFFER = 60
PREFETCH_BUFFER = 6

def preprocess_client_dataset(dataset):
    def batch_format_fn(element):
        return collections.OrderedDict(
            x=tf.reshape(element['x'], [-1, 336]),
            y=tf.reshape(element['y'], [-1, 1]))
    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
        BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)


preprocessed_example_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(example_client_dataset))


sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_client_dataset)))

sample_batch

OrderedDict([('x',
              array([[0.051, 0.051, 0.028, ..., 0.19 , 0.223, 0.219],
                     [0.055, 0.053, 0.052, ..., 0.022, 0.298, 0.783],
                     [0.052, 0.049, 0.   , ..., 0.005, 0.005, 0.037],
                     ...,
                     [0.   , 0.   , 0.034, ..., 0.022, 0.022, 0.032],
                     [0.   , 0.   , 0.   , ..., 0.005, 0.021, 0.057],
                     [0.05 , 0.05 , 0.002, ..., 0.161, 0.22 , 0.155]], dtype=float32)),
             ('y',
              array([[0.074],
                     [0.743],
                     [0.055],
                     [0.005],
                     [0.006],
                     [0.022],
                     [0.685],
                     [0.589],
                     [0.022],
                     [0.078],
                     [0.054],
                     [0.177]], dtype=float32))])

In [21]:
import random

NUM_CLIENTS = 15  # Replace with desired number of clients
all_clients = datan1['LCLid'].unique()
sample_clients = random.sample(all_clients.tolist(), NUM_CLIENTS)

sample_clients_list = sample_clients
sample_clients_list

['MAC000089',
 'MAC000028',
 'MAC000020',
 'MAC000177',
 'MAC000128',
 'MAC000019',
 'MAC000140',
 'MAC000072',
 'MAC000091',
 'MAC000092',
 'MAC000056',
 'MAC000182',
 'MAC000078',
 'MAC000086',
 'MAC000126']

In [22]:
# Iterate over unique LCLids in the dataframe
client_datasets_12 = {}
for LCLid in sample_clients_list:
    # Filter the dataframe for the current LCLid
    client_data = datan1[datan1['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)

    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))

    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))

    # Store the preprocessed dataset in the dictionary with LCLid as the key
    client_datasets_12[LCLid] = preprocessed_client_dataset

    print("Client dataset for LCLid", LCLid)
    print(sam_batch)

Client dataset for LCLid MAC000089
OrderedDict([('x', array([[0.039, 0.051, 0.054, ..., 0.035, 0.04 , 0.065],
       [0.053, 0.054, 0.037, ..., 0.043, 0.04 , 0.044],
       [0.042, 0.043, 0.037, ..., 0.037, 0.038, 0.037],
       ...,
       [0.461, 0.447, 0.264, ..., 0.469, 0.437, 0.439],
       [0.054, 0.037, 0.037, ..., 0.04 , 0.044, 0.063],
       [0.216, 0.234, 0.249, ..., 0.063, 0.174, 0.431]], dtype=float32)), ('y', array([[0.038],
       [0.063],
       [0.056],
       [0.494],
       [0.038],
       [0.431],
       [0.491],
       [0.037],
       [0.065],
       [0.091],
       [0.039],
       [0.226]], dtype=float32))])
Client dataset for LCLid MAC000028
OrderedDict([('x', array([[0.087, 0.091, 0.077, ..., 0.114, 0.112, 0.103],
       [0.075, 0.084, 0.098, ..., 0.096, 0.084, 0.107],
       [0.083, 0.081, 0.09 , ..., 0.14 , 0.07 , 0.08 ],
       ...,
       [0.081, 0.092, 0.088, ..., 0.124, 0.126, 0.11 ],
       [0.156, 0.196, 0.094, ..., 0.235, 0.264, 0.24 ],
       [0.196, 0.

In [23]:
# Iterate over unique LCLids in the dataframe
client_datasets_13 = {}
for LCLid in sample_clients_list:
    # Filter the dataframe for the current LCLid
    client_data = datan2[datan2['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)

    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))

    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))

    # Store the preprocessed dataset in the dictionary with LCLid as the key
    client_datasets_13[LCLid] = preprocessed_client_dataset



In [24]:
def make_federated_data(client_datasets, sample_clients_list):
    return [
        client_datasets[x] for x in sample_clients_list
    ]

In [25]:
federated_train_data_12 = make_federated_data(client_datasets_12, sample_clients)
federated_train_data_13 = make_federated_data(client_datasets_13, sample_clients)

In [26]:
print(f'Number of client datasets: {len(federated_train_data_12)}')
print(f'First dataset: {federated_train_data_12[0]}')
print(f'Second dataset: {federated_train_data_12[1]}')

Number of client datasets: 15
First dataset: <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>
Second dataset: <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>


In [27]:
preprocessed_example_client_dataset.element_spec

OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)),
             ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])

In [28]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(336,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    return model

In [29]:
def model_fn():
    keras_model = create_model()
    loss = tf.keras.losses.MeanSquaredError()
    tff_model = tff.learning.models.from_keras_model(
        keras_model,
        input_spec=preprocessed_example_client_dataset.element_spec,
        loss=loss,
        metrics=[tf.keras.metrics.MeanAbsoluteError()]
    )
    return tff_model

In [30]:
model = model_fn()
print(model)

In [31]:
# training starts
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [32]:
print(training_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[336,64],
      float32[64],
      float32[64,32],
      float32[32],
      float32[32,8],
      float32[8],
      float32[8,1],
      float32[1]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64,
    float32[336,64],
    float32[64],
    float32[64,32],
    float32[32],
    float32[32,8],
    float32[8],
    float32[8,1],
    float32[1]
  >
>@SERVER)


In [33]:
train_state = training_process.initialize()

In [34]:
federated_train_data_12

[<_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, n

In [35]:
federated_train_data_12

[<_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, n

In [36]:
result = training_process.next(train_state, federated_train_data_12)
train_state = result.state
train_metrics = result.metrics
print('round  1, metrics={}'.format(train_metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.07034567), ('loss', 0.0154793635), ('num_examples', 1289610), ('num_batches', 107472)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [ ]:
NUM_ROUNDS = 21
for round_num in range(2, NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data_12)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.06416719), ('loss', 0.01362201), ('num_examples', 1289610), ('num_batches', 107472)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.061836798), ('loss', 0.012987587), ('num_examples', 1289610), ('num_batches', 107472)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.060558915), ('loss', 0.012657963), ('num_examples', 1289610), ('num_batches', 107472)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('f

In [ ]:
import plotly.graph_objects as go

# Data for Cluster - 08, Batch Size = 24, Shuffle Buffer = 120, Prefetch Buffer = 12
rounds_1 = list(range(1, 16))
mae_1 = [
    0.07406851, 0.06911341, 0.06588133, 0.06381104, 0.062328234,
    0.061173238, 0.060279164, 0.059583493, 0.059030402, 0.058575064,
    0.05819059, 0.05786198, 0.05756866, 0.05733508, 0.057126228
]

# Data for Cluster - 08, Batch Size = 12, Shuffle Buffer = 60, Prefetch Buffer = 6
rounds_2 = list(range(1, 16))
mae_2 = [
    0.070309095, 0.06450078, 0.061607864, 0.060092628, 0.059076704,
    0.05836942, 0.057809025, 0.057371534, 0.05702302, 0.056733135,
    0.056459244, 0.056229576, 0.056033693, 0.055863913, 0.055711426
]

# Creating trace for Cluster - 08, Batch Size = 24, Shuffle Buffer = 120, Prefetch Buffer = 12
trace1 = go.Scatter(
    x=rounds_1,
    y=mae_1,
    mode='lines',
    name='Batch Size = 24, Shuffle Buffer = 120, Prefetch Buffer = 12'
)

# Creating trace for Cluster - 08, Batch Size = 12, Shuffle Buffer = 60, Prefetch Buffer = 6
trace2 = go.Scatter(
    x=rounds_2,
    y=mae_2,
    mode='lines',
    name='Batch Size = 12, Shuffle Buffer = 60, Prefetch Buffer = 6'
)

# Creating data list
data = [trace1, trace2]

# Creating layout
layout = go.Layout(
    title='Mean Absolute Error (MAE) vs. Federated Rounds with Learning Rate - 0.02',
    xaxis=dict(title='Federated Rounds'),
    yaxis=dict(title='MAE')
)

# Creating figure
fig = go.Figure(data=data, layout=layout)

# Displaying the graph
fig.show()


In [ ]:
import plotly.graph_objects as go

# Data for Batch Size = 12, Shuffle Buffer = 60, Prefetch Buffer = 6, Learning Rate = 0.02
rounds_1 = list(range(1, 16))
mae_1 = [
    0.070309095, 0.06450078, 0.061607864, 0.060092628, 0.059076704,
    0.05836942, 0.057809025, 0.057371534, 0.05702302, 0.056733135,
    0.056459244, 0.056229576, 0.056033693, 0.055863913, 0.055711426
]

# Data for Batch Size = 12, Shuffle Buffer = 60, Prefetch Buffer = 6, Learning Rate = 0.01
rounds_2 = list(range(1, 16))
mae_2 = [
    0.06575257, 0.059883647, 0.05776895, 0.056706358, 0.056026068,
    0.055522144, 0.05512807, 0.05480615, 0.054533303, 0.05431394,
    0.054116458, 0.053937282, 0.053791393, 0.05366001, 0.05353039
]

# Creating trace for Batch Size = 12, Shuffle Buffer = 60, Prefetch Buffer = 6, Learning Rate = 0.02
trace1 = go.Scatter(
    x=rounds_1,
    y=mae_1,
    mode='lines',
    name='Learning Rate = 0.02'
)

# Creating trace for Batch Size = 12, Shuffle Buffer = 60, Prefetch Buffer = 6, Learning Rate = 0.01
trace2 = go.Scatter(
    x=rounds_2,
    y=mae_2,
    mode='lines',
    name='Learning Rate = 0.01'
)

# Creating data list
data = [trace1, trace2]

# Creating layout
layout = go.Layout(
    title='Mean Absolute Error (MAE) vs. Federated Rounds',
    xaxis=dict(title='Federated Rounds'),
    yaxis=dict(title='MAE')
)

# Creating figure
fig = go.Figure(data=data, layout=layout)

# Displaying the graph
fig.show()


In [ ]:
# Filter data for the specified time period
start_date = '2014-01-01'
end_date = '2014-02-28'
filtered_data = data[(data['DateTime'] >= start_date) & (data['DateTime'] <= end_date)]

# Calculate the average KWH/hh for each LCLid
average_kwh = filtered_data.groupby('LCLid')['KWH/hh'].mean()

# Find the LCLids with the lowest, highest, and medium average KWH/hh
lowest_avg_lclid = average_kwh.idxmin()
highest_avg_lclid = average_kwh.idxmax()
medium_avg_lclid = average_kwh.sort_values().index[len(average_kwh) // 2]

lowest_avg_lclid, highest_avg_lclid, medium_avg_lclid

('MAC002612', 'MAC003160', 'MAC001238')

In [ ]:
# Assuming your data is stored in a DataFrame called 'data'
# Calculate the average KWH/hh for each LCLid
average_kwh = filtered_data.groupby('LCLid')['KWH/hh'].mean()

# Print all LCLids with their corresponding average KWH/hh
for lclid, avg_kwh in average_kwh.items():
    print(f"LCLid: {lclid}, Average KWH/hh: {avg_kwh}")

LCLid: MAC000005, Average KWH/hh: 0.11030821672048798
LCLid: MAC000014, Average KWH/hh: 0.16143052050269302
LCLid: MAC000019, Average KWH/hh: 0.16004305712235378
LCLid: MAC000020, Average KWH/hh: 0.16354538923573736
LCLid: MAC000052, Average KWH/hh: 0.23747900929314675
LCLid: MAC000056, Average KWH/hh: 0.1274879799425906
LCLid: MAC000072, Average KWH/hh: 0.10833871553641908
LCLid: MAC000078, Average KWH/hh: 0.12667276627197704
LCLid: MAC000086, Average KWH/hh: 0.17963329756010046
LCLid: MAC000089, Average KWH/hh: 0.2536597272792534
LCLid: MAC000091, Average KWH/hh: 0.14695328789076537
LCLid: MAC000092, Average KWH/hh: 0.0717108001435235
LCLid: MAC000099, Average KWH/hh: 0.09875385722999641
LCLid: MAC000107, Average KWH/hh: 0.1343151470926059
LCLid: MAC000122, Average KWH/hh: 0.12562002145676354
LCLid: MAC000126, Average KWH/hh: 0.11438356655902403
LCLid: MAC000128, Average KWH/hh: 0.10685991386494252
LCLid: MAC000140, Average KWH/hh: 0.11733405091496234
LCLid: MAC000167, Average KWH/hh

In [ ]:
tdata_lavg = data[data['LCLid'] == lowest_avg_lclid]
tdata_lavg

,LCLid,stdorToU,DateTime,KWH/hh,cluster
7689742,MAC002612,Std,2012-07-10 12:30:00,0.157,8
7689743,MAC002612,Std,2012-07-10 13:00:00,0.112,8
7689744,MAC002612,Std,2012-07-10 13:30:00,0.021,8
7689745,MAC002612,Std,2012-07-10 14:00:00,0.006,8
7689746,MAC002612,Std,2012-07-10 14:30:00,0.022,8
...,...,...,...,...,...
7718435,MAC002612,Std,2014-02-27 22:30:00,0.000,8
7718436,MAC002612,Std,2014-02-27 23:00:00,0.000,8
7718437,MAC002612,Std,2014-02-27 23:30:00,0.000,8
7718438,MAC002612,Std,2014-02-28 00:00:00,0.000,8


In [ ]:
tdata_havg = data[data['LCLid'] == highest_avg_lclid]
tdata_havg

,LCLid,stdorToU,DateTime,KWH/hh,cluster
9135597,MAC003160,Std,2012-07-20 15:30:00,0.237,8
9135598,MAC003160,Std,2012-07-20 16:00:00,0.347,8
9135599,MAC003160,Std,2012-07-20 16:30:00,0.638,8
9135600,MAC003160,Std,2012-07-20 17:00:00,0.339,8
9135601,MAC003160,Std,2012-07-20 17:30:00,0.074,8
...,...,...,...,...,...
9163805,MAC003160,Std,2014-02-27 22:30:00,0.005,8
9163806,MAC003160,Std,2014-02-27 23:00:00,0.006,8
9163807,MAC003160,Std,2014-02-27 23:30:00,0.044,8
9163808,MAC003160,Std,2014-02-28 00:00:00,0.048,8


In [ ]:
tdata_mavg = data[data['LCLid'] == medium_avg_lclid]
tdata_mavg

,LCLid,stdorToU,DateTime,KWH/hh,cluster
4083563,MAC001238,Std,2012-05-14 09:00:00,0.306,8
4083564,MAC001238,Std,2012-05-14 09:30:00,0.103,8
4083565,MAC001238,Std,2012-05-14 10:00:00,0.104,8
4083566,MAC001238,Std,2012-05-14 10:30:00,0.103,8
4083567,MAC001238,Std,2012-05-14 11:00:00,0.101,8
...,...,...,...,...,...
4114902,MAC001238,Std,2014-02-27 22:30:00,0.061,8
4114903,MAC001238,Std,2014-02-27 23:00:00,0.111,8
4114904,MAC001238,Std,2014-02-27 23:30:00,0.068,8
4114905,MAC001238,Std,2014-02-28 00:00:00,0.064,8


In [ ]:
filtered_data44 = data[data['LCLid'] == 'MAC000429']

In [ ]:
# Filter the DataFrame to include readings from Jan 01, 2014, to Jan 31, 2014
teststart_date1 = pd.to_datetime('2014-01-01')
testend_date1 = pd.to_datetime('2014-01-31')
test_data1 = filtered_data44[(filtered_data44['DateTime'] >= teststart_date1) & (filtered_data44['DateTime'] <= testend_date1)]

teststart_date2 = pd.to_datetime('2014-01-25')
testend_date2 = pd.to_datetime('2014-02-28')
test_data2 = filtered_data44[(filtered_data44['DateTime'] >= teststart_date2) & (filtered_data44['DateTime'] <= testend_date2)]


teststart_date3 = pd.to_datetime('2014-01-15')
testend_date3 = pd.to_datetime('2014-01-29')
test_data3 = filtered_data44[(filtered_data44['DateTime'] >= teststart_date3) & (filtered_data44['DateTime'] <= testend_date3)]

teststart_date4 = pd.to_datetime('2014-01-22')
testend_date4 = pd.to_datetime('2014-02-05')
test_data4 = filtered_data44[(filtered_data44['DateTime'] >= teststart_date4) & (filtered_data44['DateTime'] <= testend_date4)]

teststart_date5 = pd.to_datetime('2014-01-29')
testend_date5 = pd.to_datetime('2014-02-05')
test_data5 = filtered_data44[(filtered_data44['DateTime'] >= teststart_date5) & (filtered_data44['DateTime'] <= testend_date5)]

# teststart_date6 = pd.to_datetime('2014-01-06')
# testend_date6 = pd.to_datetime('2014-01-14')
# test_data6 = filtered_data44[(filtered_data44['DateTime'] >= teststart_date6) & (filtered_data44['DateTime'] <= testend_date6)]

# teststart_date7 = pd.to_datetime('2014-01-07')
# testend_date7 = pd.to_datetime('2014-01-15')
# test_data7 = filtered_data44[(filtered_data44['DateTime'] >= teststart_date7) & (filtered_data44['DateTime'] <= testend_date7)]

In [ ]:
test_data1

,LCLid,stdorToU,DateTime,KWH/hh,cluster
1560870,MAC000429,Std,2014-01-01 00:00:00,0.183,8
1560871,MAC000429,Std,2014-01-01 00:30:00,0.111,8
1560872,MAC000429,Std,2014-01-01 01:00:00,0.584,8
1560873,MAC000429,Std,2014-01-01 01:30:00,0.085,8
1560874,MAC000429,Std,2014-01-01 02:00:00,0.086,8
...,...,...,...,...,...
1562307,MAC000429,Std,2014-01-30 22:00:00,0.083,8
1562308,MAC000429,Std,2014-01-30 22:30:00,0.050,8
1562309,MAC000429,Std,2014-01-30 23:00:00,0.073,8
1562310,MAC000429,Std,2014-01-30 23:30:00,0.096,8


In [ ]:
# Select unique LCLids
lclid_lt1 = test_data1['LCLid'].unique()
sel_lclids1 = lclid_lt1[:1]
# Filter data for the selected LCLids
ft_data1 = test_data1[test_data1['LCLid'].isin(sel_lclids1)]

# Select unique LCLids
lclid_lt2 = test_data2['LCLid'].unique()
sel_lclids2 = lclid_lt2[:1]
# Filter data for the selected LCLids
ft_data2 = test_data2[test_data2['LCLid'].isin(sel_lclids2)]

# Select unique LCLids
lclid_lt3 = test_data3['LCLid'].unique()
sel_lclids3 = lclid_lt3[:1]
# Filter data for the selected LCLids
ft_data3 = test_data3[test_data3['LCLid'].isin(sel_lclids3)]

# Select unique LCLids
lclid_lt4 = test_data4['LCLid'].unique()
sel_lclids4 = lclid_lt4[:1]
# Filter data for the selected LCLids
ft_data4 = test_data4[test_data4['LCLid'].isin(sel_lclids4)]

# Select unique LCLids
lclid_lt5 = test_data5['LCLid'].unique()
sel_lclids5 = lclid_lt5[:1]
# Filter data for the selected LCLids
ft_data5 = test_data5[test_data5['LCLid'].isin(sel_lclids5)]

# # Select unique LCLids
# lclid_lt6 = test_data6['LCLid'].unique()
# sel_lclids6 = lclid_lt6[:1]
# # Filter data for the selected LCLids
# ft_data6 = test_data6[test_data6['LCLid'].isin(sel_lclids6)]

# # Select unique LCLids
# lclid_lt7 = test_data7['LCLid'].unique()
# sel_lclids7 = lclid_lt7[:1]
# # Filter data for the selected LCLids
# ft_data7 = test_data7[test_data7['LCLid'].isin(sel_lclids7)]

In [ ]:
ft_data1.reset_index(drop=True, inplace=True)
ft_data2.reset_index(drop=True, inplace=True)
ft_data3.reset_index(drop=True, inplace=True)
ft_data4.reset_index(drop=True, inplace=True)
ft_data5.reset_index(drop=True, inplace=True)
# ft_data6.reset_index(drop=True, inplace=True)
# ft_data7.reset_index(drop=True, inplace=True)

In [ ]:
ft_data1['KWH/hh'] = ft_data1['KWH/hh'].astype(np.float32)
ft_data2['KWH/hh'] = ft_data2['KWH/hh'].astype(np.float32)
ft_data3['KWH/hh'] = ft_data3['KWH/hh'].astype(np.float32)
ft_data4['KWH/hh'] = ft_data4['KWH/hh'].astype(np.float32)
ft_data5['KWH/hh'] = ft_data5['KWH/hh'].astype(np.float32)
# ft_data6['KWH/hh'] = ft_data6['KWH/hh'].astype(np.float32)
# ft_data7['KWH/hh'] = ft_data7['KWH/hh'].astype(np.float32)

In [ ]:
ft_data1 = ft_data1.drop('cluster', axis=1)
ft_data1 = ft_data1.drop('stdorToU', axis=1)

ft_data2 = ft_data2.drop('cluster', axis=1)
ft_data2 = ft_data2.drop('stdorToU', axis=1)

ft_data3 = ft_data3.drop('cluster', axis=1)
ft_data3 = ft_data3.drop('stdorToU', axis=1)

ft_data4 = ft_data4.drop('cluster', axis=1)
ft_data4 = ft_data4.drop('stdorToU', axis=1)

ft_data5 = ft_data5.drop('cluster', axis=1)
ft_data5 = ft_data5.drop('stdorToU', axis=1)

# ft_data6 = ft_data6.drop('cluster', axis=1)
# ft_data6 = ft_data6.drop('stdorToU', axis=1)

# ft_data7 = ft_data7.drop('cluster', axis=1)
# ft_data7 = ft_data7.drop('stdorToU', axis=1)


In [ ]:
ft_data1['DateTime'] = pd.to_datetime(ft_data1.DateTime).dt.tz_localize(None)
for i in range(len(ft_data1)):
  ft_data1['DateTime'][i]=ft_data1['DateTime'][i].timestamp()

ft_data2['DateTime'] = pd.to_datetime(ft_data2.DateTime).dt.tz_localize(None)
for i in range(len(ft_data2)):
  ft_data2['DateTime'][i]=ft_data2['DateTime'][i].timestamp()

ft_data3['DateTime'] = pd.to_datetime(ft_data3.DateTime).dt.tz_localize(None)
for i in range(len(ft_data3)):
  ft_data3['DateTime'][i]=ft_data3['DateTime'][i].timestamp()

ft_data4['DateTime'] = pd.to_datetime(ft_data4.DateTime).dt.tz_localize(None)
for i in range(len(ft_data4)):
  ft_data4['DateTime'][i]=ft_data4['DateTime'][i].timestamp()

ft_data5['DateTime'] = pd.to_datetime(ft_data5.DateTime).dt.tz_localize(None)
for i in range(len(ft_data5)):
  ft_data5['DateTime'][i]=ft_data5['DateTime'][i].timestamp()

# ft_data6['DateTime'] = pd.to_datetime(ft_data6.DateTime).dt.tz_localize(None)
# for i in range(len(ft_data6)):
#   ft_data6['DateTime'][i]=ft_data6['DateTime'][i].timestamp()

# ft_data7['DateTime'] = pd.to_datetime(ft_data7.DateTime).dt.tz_localize(None)
# for i in range(len(ft_data7)):
#   ft_data7['DateTime'][i]=ft_data7['DateTime'][i].timestamp()

<ipython-input-268-38af0283acac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ft_data1['DateTime'][i]=ft_data1['DateTime'][i].timestamp()
<ipython-input-268-38af0283acac>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ft_data2['DateTime'][i]=ft_data2['DateTime'][i].timestamp()
<ipython-input-268-38af0283acac>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ft_data3['DateTime'][i]=ft_data3['DateTime'][i].timestamp()
<ipython-inp

In [ ]:
ft_data1.dtypes

LCLid        object
DateTime     object
KWH/hh      float32
dtype: object

In [ ]:
ft_data1['DateTime'] = ft_data1['DateTime'].astype(np.float32)
ft_data2['DateTime'] = ft_data2['DateTime'].astype(np.float32)
ft_data3['DateTime'] = ft_data3['DateTime'].astype(np.float32)
ft_data4['DateTime'] = ft_data4['DateTime'].astype(np.float32)
ft_data5['DateTime'] = ft_data5['DateTime'].astype(np.float32)
# ft_data6['DateTime'] = ft_data6['DateTime'].astype(np.float32)
# ft_data7['DateTime'] = ft_data7['DateTime'].astype(np.float32)



In [ ]:
ft_data1

,LCLid,DateTime,KWH/hh
0,MAC000429,1.388534e+09,0.183
1,MAC000429,1.388536e+09,0.111
2,MAC000429,1.388538e+09,0.584
3,MAC000429,1.388540e+09,0.085
4,MAC000429,1.388542e+09,0.086
...,...,...,...
1437,MAC000429,1.391119e+09,0.083
1438,MAC000429,1.391121e+09,0.050
1439,MAC000429,1.391123e+09,0.073
1440,MAC000429,1.391125e+09,0.096


In [ ]:
# Sort the data by 'LCLid' and 'DateTime'
ft_data1.sort_values(['LCLid', 'DateTime'], inplace=True)
ft_data2.sort_values(['LCLid', 'DateTime'], inplace=True)
ft_data3.sort_values(['LCLid', 'DateTime'], inplace=True)
ft_data4.sort_values(['LCLid', 'DateTime'], inplace=True)
ft_data5.sort_values(['LCLid', 'DateTime'], inplace=True)
# ft_data6.sort_values(['LCLid', 'DateTime'], inplace=True)
# ft_data7.sort_values(['LCLid', 'DateTime'], inplace=True)

In [ ]:
test_client = ft_data1['LCLid'].unique()

test_client

array(['MAC000429'], dtype=object)

In [ ]:
test_dataset1 = {}

for LCLid in test_client:
    # Filter the dataframe for the current LCLid
    client_data = ft_data1[ft_data1['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)

    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))

    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))

    # Store the preprocessed dataset in the dictionary with LCLid as the key
    test_dataset1[LCLid] = preprocessed_client_dataset

    print("Client dataset for LCLid", LCLid)
    print(sam_batch)

Client dataset for LCLid MAC000429
OrderedDict([('x', array([[0.085, 0.086, 0.076, ..., 0.227, 0.115, 0.069],
       [0.353, 0.154, 0.171, ..., 0.318, 0.135, 0.205],
       [0.042, 0.086, 0.083, ..., 0.045, 0.076, 0.086],
       ...,
       [0.066, 0.086, 0.096, ..., 0.053, 0.044, 0.11 ],
       [0.087, 0.084, 0.076, ..., 0.06 , 0.088, 0.067],
       [0.481, 0.452, 0.382, ..., 0.137, 0.268, 0.053]], dtype=float32)), ('y', array([[0.088],
       [0.168],
       [0.084],
       [0.053],
       [0.045],
       [0.221],
       [0.389],
       [0.089],
       [0.078],
       [0.05 ],
       [0.105],
       [0.059],
       [0.328],
       [0.11 ],
       [0.097],
       [0.995],
       [0.054],
       [0.109],
       [0.044],
       [0.07 ]], dtype=float32))])


In [ ]:
test_dataset2 = {}

for LCLid in test_client:
    # Filter the dataframe for the current LCLid
    client_data = ft_data2[ft_data2['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)

    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))

    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))

    # Store the preprocessed dataset in the dictionary with LCLid as the key
    test_dataset2[LCLid] = preprocessed_client_dataset


In [ ]:
test_dataset3 = {}

for LCLid in test_client:
    # Filter the dataframe for the current LCLid
    client_data = ft_data3[ft_data3['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)

    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))

    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))

    # Store the preprocessed dataset in the dictionary with LCLid as the key
    test_dataset3[LCLid] = preprocessed_client_dataset


In [ ]:
test_dataset4 = {}

for LCLid in test_client:
    # Filter the dataframe for the current LCLid
    client_data = ft_data4[ft_data4['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)

    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))

    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))

    # Store the preprocessed dataset in the dictionary with LCLid as the key
    test_dataset4[LCLid] = preprocessed_client_dataset

In [ ]:
test_dataset5 = {}

for LCLid in test_client:
    # Filter the dataframe for the current LCLid
    client_data = ft_data5[ft_data5['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)

    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))

    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))

    # Store the preprocessed dataset in the dictionary with LCLid as the key
    test_dataset5[LCLid] = preprocessed_client_dataset

In [ ]:
# test_dataset6 = {}

# for LCLid in test_client:
#     # Filter the dataframe for the current LCLid
#     client_data = ft_data6[ft_data6['LCLid'] == LCLid]

#     clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)

#     # Create the client dataset for the current LCLid
#     preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))

#     # Extract a sample batch from the preprocessed dataset
#     sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))

#     # Store the preprocessed dataset in the dictionary with LCLid as the key
#     test_dataset6[LCLid] = preprocessed_client_dataset

In [ ]:
# test_dataset7 = {}

# for LCLid in test_client:
#     # Filter the dataframe for the current LCLid
#     client_data = ft_data7[ft_data7['LCLid'] == LCLid]

#     clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)

#     # Create the client dataset for the current LCLid
#     preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))

#     # Extract a sample batch from the preprocessed dataset
#     sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))

#     # Store the preprocessed dataset in the dictionary with LCLid as the key
#     test_dataset7[LCLid] = preprocessed_client_dataset

In [ ]:
# Make federated test data
federated_test_data1 = make_federated_data(test_dataset1, test_client)
federated_test_data2 = make_federated_data(test_dataset2, test_client)
federated_test_data3 = make_federated_data(test_dataset3, test_client)
federated_test_data4 = make_federated_data(test_dataset4, test_client)
federated_test_data5 = make_federated_data(test_dataset5, test_client)
# federated_test_data6 = make_federated_data(test_dataset6, test_client)
# federated_test_data7 = make_federated_data(test_dataset7, test_client)

In [ ]:
evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)

In [ ]:
print(evaluation_process.next.type_signature.formatted_representation())

(<
  state=<
    global_model_weights=<
      trainable=<
        float32[336,64],
        float32[64],
        float32[64,32],
        float32[32],
        float32[32,1],
        float32[1]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <>,
      <
        mean_absolute_error=<
          float32,
          float32
        >,
        loss=<
          float32,
          float32
        >,
        num_examples=<
          int64
        >,
        num_batches=<
          int64
        >
      >
    >,
    aggregator=<
      value_sum_process=<>,
      weight_sum_process=<>
    >,
    finalizer=<>
  >@SERVER,
  client_data={<
    x=float32[?,336],
    y=float32[?,1]
  >*}@CLIENTS
> -> <
  state=<
    global_model_weights=<
      trainable=<
        float32[336,64],
        float32[64],
        float32[64,32],
        float32[32],
        float32[32,1],
        float32[1]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <

In [ ]:
evaluation_state = evaluation_process.initialize()
model_weights = training_process.get_model_weights(train_state)
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

In [ ]:
evaluation_output1 = evaluation_process.next(evaluation_state, federated_test_data1)
evaluation_output2 = evaluation_process.next(evaluation_state, federated_test_data2)
evaluation_output3 = evaluation_process.next(evaluation_state, federated_test_data3)
evaluation_output4 = evaluation_process.next(evaluation_state, federated_test_data4)
evaluation_output5 = evaluation_process.next(evaluation_state, federated_test_data5)
# evaluation_output6 = evaluation_process.next(evaluation_state, federated_test_data6)
# evaluation_output7 = evaluation_process.next(evaluation_state, federated_test_data7)

In [ ]:
loss_values_1 = []

# Append MAE values to the list
loss_values_1.append(evaluation_output1.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
loss_values_1.append(evaluation_output2.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
loss_values_1.append(evaluation_output3.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
loss_values_1.append(evaluation_output4.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
loss_values_1.append(evaluation_output5.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
# loss_values_1.append(evaluation_output6.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
# loss_values_1.append(evaluation_output7.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])

# Print the MAE values
print(loss_values_1)

[0.11256281, 0.0347109, 0.073597565, 0.037850056, 0.03224849]


In [ ]:
NUM_ROUNDS = 16
for round_num in range(1, NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data_13)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.054316048), ('loss', 0.010862483), ('num_examples', 811935), ('num_batches', 40600)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.05395219), ('loss', 0.010799821), ('num_examples', 811935), ('num_batches', 40600)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.053744677), ('loss', 0.010755343), ('num_examples', 811935), ('num_batches', 40600)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finali

In [ ]:
evaluation_process2 = tff.learning.algorithms.build_fed_eval(model_fn)

In [ ]:
evaluation_state2 = evaluation_process2.initialize()
model_weights = training_process.get_model_weights(train_state)
evaluation_state2 = evaluation_process2.set_model_weights(evaluation_state2, model_weights)

In [ ]:
evaluation_output2_1 = evaluation_process2.next(evaluation_state2, federated_test_data1)
evaluation_output2_2 = evaluation_process2.next(evaluation_state2, federated_test_data2)
evaluation_output2_3 = evaluation_process2.next(evaluation_state2, federated_test_data3)
evaluation_output2_4 = evaluation_process2.next(evaluation_state2, federated_test_data4)
evaluation_output2_5 = evaluation_process2.next(evaluation_state2, federated_test_data5)
# evaluation_output2_6 = evaluation_process2.next(evaluation_state2, federated_test_data6)
# evaluation_output2_7 = evaluation_process2.next(evaluation_state2, federated_test_data7)

In [ ]:
loss_values_2 = []

# Append MAE values to the list
loss_values_2.append(evaluation_output2_1.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
loss_values_2.append(evaluation_output2_2.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
loss_values_2.append(evaluation_output2_3.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
loss_values_2.append(evaluation_output2_4.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
loss_values_2.append(evaluation_output2_5.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
# loss_values_2.append(evaluation_output2_6.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])
# loss_values_2.append(evaluation_output2_7.metrics['client_work']['eval']['current_round_metrics']['mean_absolute_error'])

# Print the MAE values
print(loss_values_2)

[0.09900194, 0.03780721, 0.06357957, 0.03907571, 0.03658308]
